In [75]:
from PIL import Image, ImageDraw, ImageOps
import numpy as np
from scipy.ndimage import binary_dilation
import scipy.ndimage as ndi
import cv2

# Load the original image and mask
original_image = Image.open(r'C:\Users\Shaurya\Desktop\NASH Classification\Original Images\TRAIN\1001130_43.png').convert('RGB')
mask_image = Image.open(r'C:\Users\Shaurya\Desktop\NASH Classification\Manual annotations\TRAIN\Steatosis\1001130_43.png')
tissue_image = Image.open(r'C:\Users\Shaurya\Desktop\NASH Classification\Manual annotations\TRAIN\Tissue\1001130_43.png')

# Convert images to numpy arrays
original_array = np.array(original_image)
tissue_array = np.array(tissue_image)

if np.issubdtype(tissue_array.dtype, np.bool_):
    tissue_array = tissue_array.astype(np.uint8) * 255

# Define the threshold for white pixels
threshold = 250  # Adjust this value if needed for white detection

# Create masks for the edges (top, bottom, left, right)
edge_thickness = 1  # Define thickness of the edge strip if needed

# Top edge
top_edge = np.zeros_like(tissue_array, dtype=bool)
top_edge[:edge_thickness, :] = tissue_array[:edge_thickness, :] >= threshold

# Bottom edge
bottom_edge = np.zeros_like(tissue_array, dtype=bool)
bottom_edge[-edge_thickness:, :] = tissue_array[-edge_thickness:, :] >= threshold

# Left edge
left_edge = np.zeros_like(tissue_array, dtype=bool)
left_edge[:, :edge_thickness] = tissue_array[:, :edge_thickness] >= threshold

# Right edge
right_edge = np.zeros_like(tissue_array, dtype=bool)
right_edge[:, -edge_thickness:] = tissue_array[:, -edge_thickness:] >= threshold

# Combine all edges
edges = top_edge | bottom_edge | left_edge | right_edge

# Define a 4x4 structuring element for dilation
structure = np.ones((8, 8), dtype=int)  # Use built-in int instead of np.int

# Apply dilation to the edges to match the desired thickness
dilated_edges = ndi.binary_dilation(edges, structure=structure).astype(edges.dtype)

# Print edge detection results for debugging
print(f"Dilated edges pixels: {np.sum(dilated_edges)}")

# Update the original image: set the color to black at dilated edge positions
original_array[dilated_edges] = [0, 0, 0]  # RGB black

# Print edge detection results for debugging
print(f"Top edge pixels: {np.sum(top_edge)}")
print(f"Bottom edge pixels: {np.sum(bottom_edge)}")
print(f"Left edge pixels: {np.sum(left_edge)}")
print(f"Right edge pixels: {np.sum(right_edge)}")


# Convert back to an image
output_image = Image.fromarray(original_array)

original_array = np.array(output_image)

# Detect the boundary between black and white regions in the mask
boundary_x = np.abs(np.diff(tissue_array, axis=0))[:, :-1]
boundary_y = np.abs(np.diff(tissue_array, axis=1))[:-1, :]
boundary = np.pad(boundary_x + boundary_y, ((0, 1), (0, 1)), mode='constant')
boundary = np.clip(boundary, 0, 1)  # Ensure boundary is either 0 or 1

# Define a structuring element for dilation (e.g., a 3x3 square)
structure = np.ones((4, 4), dtype=int)  # Use built-in int instead of np.int

# Apply dilation to the boundary
dilated_boundary = ndi.binary_dilation(boundary, structure=structure).astype(boundary.dtype)

# Create a new array where boundary pixels are 0, and others are unchanged from the original image
output_array = np.copy(original_array)
output_array[dilated_boundary == 1] = 0  # Set dilated boundary pixels to 0 (black)

# Convert the output array back to an image
original_image = Image.fromarray(output_array)



# Convert images to numpy arrays
original_array = np.array(original_image)
mask_array = np.array(mask_image)

# Convert mask to binary (0 and 1)
binary_mask = np.where(mask_array > 0, 1, 0).astype(np.uint8)

# Label connected components
labeled_mask, num_features = ndi.label(binary_mask)

# Pixel to micrometer conversion factor
pixel_to_um = 1.40 / 3
pixel_to_um2 = pixel_to_um ** 2

# Define colors
color_less_than_175_um2 = [0, 255, 255]  # Cyan
color_greater_than_175_um2 = [255, 165, 0]  # Orange 

# Initialize the output array with the original image
output_array = np.copy(original_array)

# Define a structuring element for dilation (e.g., a 4x4 square for a thicker boundary)
structure = np.ones((4, 4), dtype=int)

# Process each labeled region
for label in range(1, num_features + 1):
    # Create a mask for the current labeled region
    region_mask = (labeled_mask == label).astype(np.uint8)
    
    # Calculate area of the current region
    area_pixels = np.sum(region_mask)
    area_um2 = area_pixels * pixel_to_um2
    print(f"Region {label} area: {area_um2:.2f} µm²")

    # Determine color based on area
    if area_um2 > 175:
        color = color_greater_than_175_um2 
    else:
        color = color_less_than_175_um2

    # Detect the boundary between black and white regions in the region mask
    boundary_x = np.abs(np.diff(region_mask, axis=0))[:, :-1]
    boundary_y = np.abs(np.diff(region_mask, axis=1))[:-1, :]
    boundary = np.pad(boundary_x + boundary_y, ((0, 1), (0, 1)), mode='constant')
    boundary = np.clip(boundary, 0, 1)  # Ensure boundary is either 0 or 1

    # Apply dilation to the boundary to create a thicker boundary
    dilated_boundary = ndi.binary_dilation(boundary, structure=structure).astype(boundary.dtype)

    # Apply the color to the boundary in the output image
    for c in range(3):  # For each color channel (R, G, B)
        output_array[:, :, c][dilated_boundary == 1] = color[c]

# Convert the output array back to an image
output_image_colored = Image.fromarray(output_array)

# Display the resulting image
output_image_colored.show()

output_image_colored.save('Manual Annotations.png')

Dilated edges pixels: 3974
Top edge pixels: 0
Bottom edge pixels: 416
Left edge pixels: 32
Right edge pixels: 355
Region 1 area: 10.24 µm²
Region 2 area: 8.06 µm²
Region 3 area: 8.06 µm²
Region 4 area: 29.62 µm²
Region 5 area: 10.24 µm²
Region 6 area: 10.24 µm²
Region 7 area: 12.85 µm²
Region 8 area: 15.03 µm²
Region 9 area: 10.45 µm²
